In [1]:
%load_ext autoreload
%autoreload 2

import os
import time
import sys

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *

Setting environment for AWS compute node


No vtk
File does not exist: /shared/CSHL_data_processed/MD585/MD585_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD585/MD585_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD585/MD585_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD585/MD585_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD589/MD589_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD589/MD589_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD589/MD589_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD590/MD590_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD590/MD590_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD590/MD590_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD590/MD590_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD591/MD591_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD591/MD591_sorted_filenames.txt
File does not exist: 

In [2]:
stack = 'MD589'
_, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)

In [3]:
filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]

In [4]:
from distributed_utilities import *

In [ ]:
run_distributed("%(script)s %(stack)s %(input_dir)s %(elastix_output_dir)s \'%%(kwargs_str)s\'" % \
                {'stack': stack,
                'script': os.path.join(REPO_DIR, 'preprocess', 'align_consecutive_v2.py'),
                'input_dir': os.path.join(RAW_DATA_DIR, stack),
                'elastix_output_dir': os.path.join(DATA_DIR, stack, stack+'_elastix_output')},
#                 kwargs_list=[{'prev_fn': filenames[i-1], 'curr_fn': filenames[i]} for i in range(1, len(filenames))],
                kwargs_list=[{'prev_fn': filenames[i-1], 'curr_fn': filenames[i]} for i in range(1, 5)],
                argument_type='list',
               cluster_size=2)

Setting environment for AWS compute node
Scaling cluster: cfncluster-yuncongCluster-ComputeFleet-132HFSI0PQOU6 Size: 2


In [25]:
json.loads("""[{\"curr_fn\": \"MD589-IHC1-2015.07.30-21.30.07_MD589_2_0002\", \"prev_fn\": \"MD589-N1-2015.07.30-16.19.59_MD589_2_0002\"}, {\"curr_fn\": \"MD589-N1-2015.07.30-16.19.59_MD589_3_0003\", \"prev_fn\": \"MD589-IHC1-2015.07.30-21.30.07_MD589_2_0002\"}]""")

[{u'curr_fn': u'MD589-IHC1-2015.07.30-21.30.07_MD589_2_0002',
  u'prev_fn': u'MD589-N1-2015.07.30-16.19.59_MD589_2_0002'},
 {u'curr_fn': u'MD589-N1-2015.07.30-16.19.59_MD589_3_0003',
  u'prev_fn': u'MD589-IHC1-2015.07.30-21.30.07_MD589_2_0002'}]